# Introduction

<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to setup with Azure account for Azure connection using TNGTLS. By using this Notebook, its possible to update account credentials to Azure cli, Register the Signer and device.

Before running this Notebook, 
1. <font><strong>docs/Azure_iot_hub_details.csv</strong></font> file should have updated with azure iot hub name.
2. Make sure CryptoAuth Trust Platform is having factory reset program

### 1.1.Tutorial Prerequisites:
    

The following code is runs on python 3.x environment. This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the above modules are already installed you can safely step over the next Tutorial step.

> <font color = green>pip install -U module_name</font>

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange> azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>
    - <font color = orange> Extension 'azure-cli-iot-ext' is already installed. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd())))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))

In [ ]:
!az extension add --name azure-cli-iot-ext

### Load the necessary modules and helper functions

In [ ]:
from helper_azure import *
from ipywidgets import *
from IPython.display import display
from tkinter import Tk, filedialog
from cryptoauthlib import *
from cryptoauthlib.iface import *
import binascii

from trustplatform import *

print("Importing modules - Successful")

#TFLXTLS device I2C address 0x6C; #TNGTLS device I2C address 0x6A; 
common_helper.connect_to_secure_element('ATECC608A', ATCAKitType.ATCA_KIT_I2C_IFACE, 0x6A)
print("Device initialization - Successful")

# Request the Serial Number
serial_number = bytearray(9)
assert atcab_read_serial_number(serial_number) == Status.ATCA_SUCCESS
device_reg_id = binascii.hexlify(serial_number).decode("utf-8").upper()
print ("Device ID: {}".format(device_reg_id))

# Verify Certificate support for Azure connection
unsupported_otp = "Rsuy5YJh"
otpcode = bytearray(32)
ATCA_ZONE_OTP = 1
assert atcab_read_zone(ATCA_ZONE_OTP, 0, 0, 0, otpcode, 32) == Status.ATCA_SUCCESS
assert unsupported_otp != otpcode[:8].decode('utf-8'), "TNGTLS connected doesnt support Azure connection! Try with another TNGTLS device"

The following steps assist to perform
    1. Azure account login
    2. Registering the Device

#### Azure account login (1. Account Login)
This helps to update user Azure credentials to Azure-cli. Azure-cli will be used for any interactions with Azure cloud from the PC.This can be skipped if the azure login update to azure cli is already done.

#### Register the Device (2. Register device)
This will register the device ID of the TNGTLS to Azure iot hub account. For registering TNGTLS device, its required to provide Manifest file and associated verification certificate. If Manifest file and verification certificates are not available, run the Trust&GO resource generation to generate. Load Manifest and Validation certs before clicking the 'Register device' button. This should register the devices in Manifest file to Azure cloud. 

### 1. Azure account login
This step helps to initialise the Azure cli tools with azure account credentials.

In [ ]:
def execute_azure_login(b):
    Azure_login.button_style = azure_account_login()
    
Azure_login = widgets.Button(description = "Azure login")
Azure_login.on_click(execute_azure_login)

### 3. Register Device ID from Manifest
This step of the tutorial registers the device ID to the Azure IOT Hub.

In [ ]:
def execute_device_register(b):
    reg_device.button_style = ''
    if not manifest_file.data or not validation_cert.data:
        print('Step2a & Step2b should be executed first before proceeding to this step\r\n')
        return None
    register_device(json.loads(manifest_file.data[0]), validation_cert.data[0])

manifest_file = FileUpload(description='Step2a. Load Manifest JSON File', accept='.json', layout=widgets.Layout(width='auto'), multiple=False)
validation_cert = FileUpload(description='Step2b. Load Validation CERT File', accept='.crt', layout=widgets.Layout(width='auto'), multiple=False)
reg_device = widgets.Button(description = "Step2. Register device", layout=widgets.Layout(width='auto'))
reg_device.on_click(execute_device_register)

display(widgets.HBox((Azure_login, widgets.VBox((manifest_file, validation_cert, reg_device)))))